# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Transform Chain: 实现快捷处理超长文本

![](../images/transform_chain.png)

In [1]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [2]:
with open("../tests/the_old_man_and_the_sea.txt") as f:
    novel_text = f.read()

In [3]:
print(novel_text)


                 老人与海　
　　　　　　　　        作者：海明威 

　　他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美东海岸向东北流动。这股暖流温度比两旁的海水高至度，最宽处达英里，呈深蓝色，非常壮观，为鱼类群集的地方。本书主人公为古巴首都哈瓦那附近小海港的渔夫，经常驶进湾流捕鱼）中一条小船上钓鱼的老人，至今已去了八十四天，一条鱼也没逮住。头四十天里，有个男孩子跟他在一起。可是，过了四十天还没捉到一条鱼，孩子的父母对他说，老人如今准是十足地"倒了血霉"，这就是说，倒霉到了极点，于是孩子听从了他们的吩咐，上了另外一条船，头一个礼拜就捕到了三条好鱼。孩子看见老人每天回来时船总是空的，感到很难受，他总是走下岸去，帮老人拿卷起的钓索，或者鱼钩和鱼叉，还有绕在桅杆上的帆。帆上用面粉袋片打了些补丁，收拢后看来象是一面标志着永远失败的旗子。 
　　老人消瘦而憔悴，脖颈上有些很深的皱纹。腮帮上有些褐斑，那是太阳在热带海面上反射的光线所引起的良性皮肤癌变。褐斑从他脸的两侧一直蔓延下去，他的双手常用绳索拉大鱼，留下了刻得很深的伤疤。但是这些伤疤中没有一块是新的。它们象无鱼可打的沙漠中被侵蚀的地方一般古老。他身上的一切都显得古老，除了那双眼睛，它们象海水一般蓝，是愉快而不肯认输的。
　　“圣地亚哥，"他们俩从小船停泊的地方爬上岸时，孩子对他说。"我又能陪你出海了。我家挣到了一点儿钱。” 
　　老人教会了这孩子捕鱼，孩子爱他。 
　　“不，”老人说。“你遇上了一条交好运的船。跟他们待下去吧。” 
　　“不过你该记得，你有一回八十七天钓不到一条鱼，跟着有三个礼拜，我们每天都逮住了大鱼。” 
　　“我记得，”老人说。“我知道你不是因为没把握才离开我的。” 
　　“是爸爸叫我走的。我是孩子，不能不听从他。” 
　　“我明白，”老人说。“这是理该如此的。” 
　　“他没多大的信心。” 
　　“是啊，”老人说。“可是我们有。可不是吗？” 
　　“对，"孩子说。"我请你到露台饭店去喝杯啤酒，然后一起把打鱼的家什带回去。” 
　　“那敢情好，”老人说。“都是打鱼人嘛。” 
　　他们坐在饭店的露台上，不少渔夫拿老人开玩笑，老人并不生气。另外一些上了些年纪的渔夫望着他，感到难受。不过他们并不流露出来，只是斯文地谈起海流，谈起他

In [4]:
len(novel_text)

47867

In [5]:
# 定义一个转换函数，输入是一个字典，输出也是一个字典。
def transform_func(inputs: dict) -> dict:
    # 从输入字典中获取"text"键对应的文本。
    text = inputs["text"]
    # 使用split方法将文本按照"\n\n"分隔为多个段落，并只取前三个，然后再使用"\n\n"将其连接起来。
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    # 返回裁剪后的文本，用"output_text"作为键。
    return {"output_text": shortened_text}

# 使用上述转换函数创建一个TransformChain对象。
# 定义输入变量为["text"]，输出变量为["output_text"]，并指定转换函数为transform_func。
transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)

In [6]:
transformed_novel = transform_chain(novel_text)

In [7]:
print(transformed_novel['text'])


                 老人与海　
　　　　　　　　        作者：海明威 

　　他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美东海岸向东北流动。这股暖流温度比两旁的海水高至度，最宽处达英里，呈深蓝色，非常壮观，为鱼类群集的地方。本书主人公为古巴首都哈瓦那附近小海港的渔夫，经常驶进湾流捕鱼）中一条小船上钓鱼的老人，至今已去了八十四天，一条鱼也没逮住。头四十天里，有个男孩子跟他在一起。可是，过了四十天还没捉到一条鱼，孩子的父母对他说，老人如今准是十足地"倒了血霉"，这就是说，倒霉到了极点，于是孩子听从了他们的吩咐，上了另外一条船，头一个礼拜就捕到了三条好鱼。孩子看见老人每天回来时船总是空的，感到很难受，他总是走下岸去，帮老人拿卷起的钓索，或者鱼钩和鱼叉，还有绕在桅杆上的帆。帆上用面粉袋片打了些补丁，收拢后看来象是一面标志着永远失败的旗子。 
　　老人消瘦而憔悴，脖颈上有些很深的皱纹。腮帮上有些褐斑，那是太阳在热带海面上反射的光线所引起的良性皮肤癌变。褐斑从他脸的两侧一直蔓延下去，他的双手常用绳索拉大鱼，留下了刻得很深的伤疤。但是这些伤疤中没有一块是新的。它们象无鱼可打的沙漠中被侵蚀的地方一般古老。他身上的一切都显得古老，除了那双眼睛，它们象海水一般蓝，是愉快而不肯认输的。
　　“圣地亚哥，"他们俩从小船停泊的地方爬上岸时，孩子对他说。"我又能陪你出海了。我家挣到了一点儿钱。” 
　　老人教会了这孩子捕鱼，孩子爱他。 
　　“不，”老人说。“你遇上了一条交好运的船。跟他们待下去吧。” 
　　“不过你该记得，你有一回八十七天钓不到一条鱼，跟着有三个礼拜，我们每天都逮住了大鱼。” 
　　“我记得，”老人说。“我知道你不是因为没把握才离开我的。” 
　　“是爸爸叫我走的。我是孩子，不能不听从他。” 
　　“我明白，”老人说。“这是理该如此的。” 
　　“他没多大的信心。” 
　　“是啊，”老人说。“可是我们有。可不是吗？” 
　　“对，"孩子说。"我请你到露台饭店去喝杯啤酒，然后一起把打鱼的家什带回去。” 
　　“那敢情好，”老人说。“都是打鱼人嘛。” 
　　他们坐在饭店的露台上，不少渔夫拿老人开玩笑，老人并不生气。另外一些上了些年纪的渔夫望着他，感到难受。不过他们并不流露出来，只是斯文地谈起海流，谈起他

In [8]:
type(transformed_novel)

dict

In [9]:
print(transformed_novel["output_text"])


                 老人与海　
　　　　　　　　        作者：海明威 

　　他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美东海岸向东北流动。这股暖流温度比两旁的海水高至度，最宽处达英里，呈深蓝色，非常壮观，为鱼类群集的地方。本书主人公为古巴首都哈瓦那附近小海港的渔夫，经常驶进湾流捕鱼）中一条小船上钓鱼的老人，至今已去了八十四天，一条鱼也没逮住。头四十天里，有个男孩子跟他在一起。可是，过了四十天还没捉到一条鱼，孩子的父母对他说，老人如今准是十足地"倒了血霉"，这就是说，倒霉到了极点，于是孩子听从了他们的吩咐，上了另外一条船，头一个礼拜就捕到了三条好鱼。孩子看见老人每天回来时船总是空的，感到很难受，他总是走下岸去，帮老人拿卷起的钓索，或者鱼钩和鱼叉，还有绕在桅杆上的帆。帆上用面粉袋片打了些补丁，收拢后看来象是一面标志着永远失败的旗子。 
　　老人消瘦而憔悴，脖颈上有些很深的皱纹。腮帮上有些褐斑，那是太阳在热带海面上反射的光线所引起的良性皮肤癌变。褐斑从他脸的两侧一直蔓延下去，他的双手常用绳索拉大鱼，留下了刻得很深的伤疤。但是这些伤疤中没有一块是新的。它们象无鱼可打的沙漠中被侵蚀的地方一般古老。他身上的一切都显得古老，除了那双眼睛，它们象海水一般蓝，是愉快而不肯认输的。
　　“圣地亚哥，"他们俩从小船停泊的地方爬上岸时，孩子对他说。"我又能陪你出海了。我家挣到了一点儿钱。” 
　　老人教会了这孩子捕鱼，孩子爱他。 
　　“不，”老人说。“你遇上了一条交好运的船。跟他们待下去吧。” 
　　“不过你该记得，你有一回八十七天钓不到一条鱼，跟着有三个礼拜，我们每天都逮住了大鱼。” 
　　“我记得，”老人说。“我知道你不是因为没把握才离开我的。” 
　　“是爸爸叫我走的。我是孩子，不能不听从他。” 
　　“我明白，”老人说。“这是理该如此的。” 
　　“他没多大的信心。” 
　　“是啊，”老人说。“可是我们有。可不是吗？” 
　　“对，"孩子说。"我请你到露台饭店去喝杯啤酒，然后一起把打鱼的家什带回去。” 
　　“那敢情好，”老人说。“都是打鱼人嘛。” 
　　他们坐在饭店的露台上，不少渔夫拿老人开玩笑，老人并不生气。另外一些上了些年纪的渔夫望着他，感到难受。不过他们并不流露出来，只是斯文地谈起海流，谈起他

In [10]:
template = """总结下面文本:

{output_text}

总结:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt, verbose=True)

In [11]:
llm_chain(transformed_novel['output_text'][:1000])



> Entering new LLMChain chain...
Prompt after formatting:
总结下面文本:


                 老人与海　
　　　　　　　　        作者：海明威 

　　他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美东海岸向东北流动。这股暖流温度比两旁的海水高至度，最宽处达英里，呈深蓝色，非常壮观，为鱼类群集的地方。本书主人公为古巴首都哈瓦那附近小海港的渔夫，经常驶进湾流捕鱼）中一条小船上钓鱼的老人，至今已去了八十四天，一条鱼也没逮住。头四十天里，有个男孩子跟他在一起。可是，过了四十天还没捉到一条鱼，孩子的父母对他说，老人如今准是十足地"倒了血霉"，这就是说，倒霉到了极点，于是孩子听从了他们的吩咐，上了另外一条船，头一个礼拜就捕到了三条好鱼。孩子看见老人每天回来时船总是空的，感到很难受，他总是走下岸去，帮老人拿卷起的钓索，或者鱼钩和鱼叉，还有绕在桅杆上的帆。帆上用面粉袋片打了些补丁，收拢后看来象是一面标志着永远失败的旗子。 
　　老人消瘦而憔悴，脖颈上有些很深的皱纹。腮帮上有些褐斑，那是太阳在热带海面上反射的光线所引起的良性皮肤癌变。褐斑从他脸的两侧一直蔓延下去，他的双手常用绳索拉大鱼，留下了刻得很深的伤疤。但是这些伤疤中没有一块是新的。它们象无鱼可打的沙漠中被侵蚀的地方一般古老。他身上的一切都显得古老，除了那双眼睛，它们象海水一般蓝，是愉快而不肯认输的。
　　“圣地亚哥，"他们俩从小船停泊的地方爬上岸时，孩子对他说。"我又能陪你出海了。我家挣到了一点儿钱。” 
　　老人教会了这孩子捕鱼，孩子爱他。 
　　“不，”老人说。“你遇上了一条交好运的船。跟他们待下去吧。” 
　　“不过你该记得，你有一回八十七天钓不到一条鱼，跟着有三个礼拜，我们每天都逮住了大鱼。” 
　　“我记得，”老人说。“我知道你不是因为没把握才离开我的。” 
　　“是爸爸叫我走的。我是孩子，不能不听从他。” 
　　“我明白，”老人说。“这是理该如此的。” 
　　“他没多大的信心。” 
　　“是啊，”老人说。“可是我们有。可不是吗？” 
　　“对，"孩子说。"我请你到露台饭店去喝杯啤酒，然后一起把打鱼的家什带回去。” 
　　“那敢情好，”老人说。“都是打鱼人嘛。” 
　　他们坐在饭

{'output_text': '\n                 老人与海\u3000\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000        作者：海明威 \n\n\u3000\u3000他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美东海岸向东北流动。这股暖流温度比两旁的海水高至度，最宽处达英里，呈深蓝色，非常壮观，为鱼类群集的地方。本书主人公为古巴首都哈瓦那附近小海港的渔夫，经常驶进湾流捕鱼）中一条小船上钓鱼的老人，至今已去了八十四天，一条鱼也没逮住。头四十天里，有个男孩子跟他在一起。可是，过了四十天还没捉到一条鱼，孩子的父母对他说，老人如今准是十足地"倒了血霉"，这就是说，倒霉到了极点，于是孩子听从了他们的吩咐，上了另外一条船，头一个礼拜就捕到了三条好鱼。孩子看见老人每天回来时船总是空的，感到很难受，他总是走下岸去，帮老人拿卷起的钓索，或者鱼钩和鱼叉，还有绕在桅杆上的帆。帆上用面粉袋片打了些补丁，收拢后看来象是一面标志着永远失败的旗子。 \n\u3000\u3000老人消瘦而憔悴，脖颈上有些很深的皱纹。腮帮上有些褐斑，那是太阳在热带海面上反射的光线所引起的良性皮肤癌变。褐斑从他脸的两侧一直蔓延下去，他的双手常用绳索拉大鱼，留下了刻得很深的伤疤。但是这些伤疤中没有一块是新的。它们象无鱼可打的沙漠中被侵蚀的地方一般古老。他身上的一切都显得古老，除了那双眼睛，它们象海水一般蓝，是愉快而不肯认输的。\n\u3000\u3000“圣地亚哥，"他们俩从小船停泊的地方爬上岸时，孩子对他说。"我又能陪你出海了。我家挣到了一点儿钱。” \n\u3000\u3000老人教会了这孩子捕鱼，孩子爱他。 \n\u3000\u3000“不，”老人说。“你遇上了一条交好运的船。跟他们待下去吧。” \n\u3000\u3000“不过你该记得，你有一回八十七天钓不到一条鱼，跟着有三个礼拜，我们每天都逮住了大鱼。” \n\u3000\u3000“我记得，”老人说。“我知道你不是因为没把握才离开我的。” \n\u3000\u3000“是爸爸叫我走的。我是孩子，不能不听从他。” \n\u3000\u3000“我明白，”老人说。“这是理该如此的。” \n\u3000\u3000“他没多大

### 使用 Sequential Chain 编排

In [12]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

In [13]:
sequential_chain.run(novel_text[:100])



> Entering new LLMChain chain...
Prompt after formatting:
总结下面文本:


                 老人与海　
　　　　　　　　        作者：海明威 

　　他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美

总结:

> Finished chain.


' 老人与海是海明威的作品，讲的是一位老人独自在墨西哥湾暖流（穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡）沿着北美洲西海岸的故事。'